## A minimalist example for recovering sparse graphs using `uGLAD`

Fitting uGLAD on a erdos-renyi random sparse graph with samples obtained from a corresponding multivariate Gaussian distribution.    

### About `uGLAD` 
Sparse graph recovery by optimizing deep unrolled networks. This work proposes `uGLAD` which is a unsupervised version of a previous `GLAD` model (GLAD: Learning Sparse Graph Recovery (ICLR 2020 - [link](<https://openreview.net/forum?id=BkxpMTEtPB>)).  

Key benefits & features:  
- Solution to Graphical Lasso: A better alternative to solve the Graphical Lasso problem as
    - The neural networks of the uGLAD enable adaptive choices of the hyperparameters which leads to better performance than the existing algorithms  
     - No need to pre-specify the sparsity related regularization hyperparameters    
    - Requires less number of iterations to converge due to neural network based acceleration of the unrolled optimization algorithm (Alternating Minimization)    
    - GPU based acceleration can be leveraged  
    - Novel `consensus` strategy which robustly handles missing values by leveraging the multi-task learning ability of the model   
    - Multi-task learning mode that solves the graphical lasso objective to recover multiple graphs with a single `uGLAD` model  
- Glasso loss function: The loss is the logdet objective of the graphical lasso `1/M(-1*log|theta|+ <S, theta>)`, where `M=num_samples, S=input covariance matrix, theta=predicted precision matrix`.  
- Ease of usability: Matches the I/O signature of `sklearn GraphicalLassoCV`, so easy to plug-in to the existing code.  

In [1]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

/home/harshx/anaconda3/envs/uGLAD


In [2]:
import torch
torch.__version__

'1.10.1'

# 1. Synthetic data convergence

In [11]:
from uGLAD.utils.prepare_data import get_data
from uGLAD.utils.metrics import reportMetrics

# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, true_theta = get_data(
    num_nodes=20, 
    sparsity=[0.2, 0.2], 
    num_samples=500, 
    batch_size=1,
    eig_offset=1, 
    w_min=0.5,
    w_max=1
)
print(f'true_theta: {true_theta.shape}, Samples {Xb.shape}')

true_theta: (1, 20, 20), Samples (1, 500, 20)


### The uGLAD model

Learning details:  
1. Initialize learnable `GLAD` parameters  
2. Run the GLAD model  
3. Get the glasso-loss  
4. Backprop  

Possible solutions if `uGLAD` does not converge:  
1. Increase number of training EPOCHS
2. Lower the learning rate    
3. Please re-run. This will run the optimization with different initializations  
4. Change the INIT_DIAG=0/1 in the `GLAD` model parameters  
5. Increase `L`, the number of unrolled iterations of `GLAD`

### Running the uGLAD-Direct mode

- Directly optimize the uGLAD model on the complete data X
- Optimizes the model to minimize the glasso-loss on X 

In [19]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb[0],
    centered=False,
    epochs=400,
    lr=0.005,
    INIT_DIAG=0,
    L=15,
    verbose=False, 
    k_fold=0,  # Direct mode
    mode='direct'
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
Processing the input table for basic compatibility check
Direct Mode
Total runtime: 11.620458126068115 secs

uGLAD: {'FDR': 0.5263157894736842, 'TPR': 1.0, 'FPR': 0.2597402597402597, 'SHD': 40, 'nnzTrue': 36, 'nnzPred': 76, 'precision': 0.47368421052631576, 'recall': 1.0, 'Fbeta': 0.6428571428571429, 'aupr': 0.9931827431827432, 'auc': 0.9981962481962482}


### Running the uGLAD-CV mode 

- Finds the best model by doing cross-fold validation on the input samples X
- Chooses the model which performs best in terms of glasso-loss on held-out data
- More conservative than the direct mode

In [13]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb[0],
    centered=False,
    epochs=400,
    lr=0.005,
    INIT_DIAG=0,
    L=15,
    verbose=False,
    k_fold=5, 
    mode='cv'
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
Processing the input table for basic compatibility check
CV mode: 5-fold
Total runtime: 74.15954613685608 secs

uGLAD: {'FDR': 0.7636363636363637, 'TPR': 1.0, 'FPR': 0.8344370860927153, 'SHD': 126, 'nnzTrue': 39, 'nnzPred': 165, 'precision': 0.23636363636363636, 'recall': 1.0, 'Fbeta': 0.38235294117647056, 'aupr': 0.9854555972977026, 'auc': 0.9952453727288165}


### Comparison with sklearn's GraphicalLassoCV

In [14]:
from sklearn.covariance import GraphicalLassoCV

model_BCD = GraphicalLassoCV().fit(Xb[0])
# Compare with theta
compare_theta_BCD = reportMetrics(
    true_theta[0], 
    model_BCD.precision_
)
print(f'BCD: {compare_theta_BCD}')

BCD: {'FDR': 0.6454545454545455, 'TPR': 1.0, 'FPR': 0.47019867549668876, 'SHD': 71, 'nnzTrue': 39, 'nnzPred': 110, 'precision': 0.35454545454545455, 'recall': 1.0, 'Fbeta': 0.5234899328859061, 'aupr': 0.9906082289803221, 'auc': 0.9969434538970964}


# 2. Handling missing values
Running `uGLAD` model in mode=`missing`:
- Leverages the multi-task learning feature of the `uGLAD` model
- Uses the novel `consensus` strategy to robustly handle the missing values

In [7]:
# Adding dropout noise to Xb
from uGLAD.utils.prepare_data import add_noise_dropout
from uGLAD.main import mean_imputation
import numpy as np

# Adding np.NaNs to introduce missing values
Xb_miss = add_noise_dropout(Xb, dropout=0.83)
# Doing mean imputation for basic statistical comparsion
B, M, D = Xb_miss.shape
Xb_mean = [] 
for b in range(B):
    X_miss = Xb_miss[b].copy()
    X_miss = X_miss.reshape(1, M, D)
    Xb_mean.append(mean_imputation(X_miss).reshape(M, D))
Xb_mean = np.array(Xb_mean)

### Running the `uGLAD-miss` model in missing data mode 

In [8]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb_miss[0],
    centered=False,
    epochs=500,
    lr=0.005,
    INIT_DIAG=0,
    L=15,
    verbose=False,
    k_fold=3,  # The number of sumsample splits
    mode='missing'
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
Handling missing data
Creating K=3 row-subsampled batches
Getting the final precision matrix using the consensus strategy
Total runtime: 40.27913165092468 secs

uGLAD: {'FDR': 0.7352941176470589, 'TPR': 0.225, 'FPR': 0.16666666666666666, 'SHD': 56, 'nnzTrue': 40, 'nnzPred': 34, 'precision': 0.2647058823529412, 'recall': 0.225, 'Fbeta': 0.24324324324324326, 'aupr': 0.2636695112229714, 'auc': 0.5360833333333334}


### Missing values: Comparison with BCD-mean
Run GraphicalLassoCV with mean imputed Xb_mean

In [9]:
from sklearn.covariance import GraphicalLassoCV

model_BCD = GraphicalLassoCV().fit(Xb_mean[0])
# Compare with theta
compare_theta_BCD = reportMetrics(
    true_theta[0], 
    model_BCD.precision_
)
print(f'BCD: {compare_theta_BCD}')

BCD: {'FDR': nan, 'TPR': 0.0, 'FPR': 0.0, 'SHD': 40, 'nnzTrue': 40, 'nnzPred': 0, 'precision': nan, 'recall': 0.0, 'Fbeta': 0.0, 'aupr': 0.21052631578947367, 'auc': 0.5}


# 3. Multi-task learning mode
- Generate synthetic data coming from graphs with varying sparsity
- Recover the batch precision matrices for the batch input data X

In [15]:
# Creating synthetic data for multi-task learning 
from uGLAD.utils.prepare_data import get_data
from uGLAD.utils.metrics import reportMetrics

# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, true_theta = get_data(
    num_nodes=20, 
    sparsity=[0.1, 0.2], 
    num_samples=500, 
    batch_size=3,
    eig_offset=1, 
    w_min=0.5,
    w_max=1
)
print(f'true_theta: {true_theta.shape}, Samples {Xb.shape}')

true_theta: (3, 20, 20), Samples (3, 500, 20)


In [16]:
# Running uGLAD in multi-task learning mode
from uGLAD import main as uG
from uGLAD.utils.metrics import summarize_compare_theta

# Initialize the model
model_uGLAD = uG.uGLAD_multitask()  

K = len(Xb)

# Fit to the data
model_uGLAD.fit(
    Xb,
    centered=False,
    epochs=200,
    lr=0.01,
    INIT_DIAG=0,
    L=15,
    verbose=False,
)

# Print the compare metrics
compare_theta_MT = []
for b in range(K):
    rM = reportMetrics(
            true_theta[b], 
            model_uGLAD.precision_[b]
        )
    print(f'Metrics for graph {b}: {rM}\n')
    compare_theta_MT.append(rM)

# Calculate the average statistics
avg_results_MT = summarize_compare_theta(compare_theta_MT, method_name='uGLAD multi-task')

Running uGLAD in multi-task mode
Processing the input table for basic compatibility check
Total runtime: 12.470507144927979 secs

Metrics for graph 0: {'FDR': 0.7966101694915254, 'TPR': 1.0, 'FPR': 0.9155844155844156, 'SHD': 141, 'nnzTrue': 36, 'nnzPred': 177, 'precision': 0.2033898305084746, 'recall': 1.0, 'Fbeta': 0.3380281690140845, 'aupr': 0.9925595238095238, 'auc': 0.9980158730158729}

Metrics for graph 1: {'FDR': 0.8245614035087719, 'TPR': 1.0, 'FPR': 0.88125, 'SHD': 141, 'nnzTrue': 30, 'nnzPred': 171, 'precision': 0.17543859649122806, 'recall': 1.0, 'Fbeta': 0.29850746268656714, 'aupr': 1.0, 'auc': 1.0}

Metrics for graph 2: {'FDR': 0.8160919540229885, 'TPR': 1.0, 'FPR': 0.8987341772151899, 'SHD': 142, 'nnzTrue': 32, 'nnzPred': 174, 'precision': 0.1839080459770115, 'recall': 1.0, 'Fbeta': 0.3106796116504854, 'aupr': 1.0, 'auc': 1.0}

Avg results for uGLAD multi-task

{'FDR': (0.8124211756744287, 0.011702530525059153),
 'FPR': (0.8985228642665352, 0.014017762856522259),
 'Fbeta':